<a href="https://colab.research.google.com/github/Mohit-Mithra/gene-coexpression-network/blob/master/hormone-gene-classification/SMOTE_embedding_space.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content


In [ ]:
%cd gdrive/My\ Drive/IITM_internship
!ls

/content/gdrive/My Drive/IITM_internship
hormone_gene_graph0.txt		visualise_embeddings.ipynb
hormone_src_tgt_truegenes.txt	word_embeddings.txt
SMOTE_embedding_space.ipynb	word_vectors.json
SMOTE_oversampled_dataset.json	word_vectors.npy


In [ ]:
import json

In [ ]:
from collections import Counter
from imblearn.over_sampling import SMOTE
from matplotlib import pyplot
import numpy as np

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
with open('hormone_src_tgt_truegenes.txt') as json_file:
    horm_gene = json.load(json_file)

In [ ]:
horm_gene.keys()

dict_keys(['Adrenaline', 'Epinephrine', 'Aldosterone', 'Androgen', 'Angiotensin ', 'Calcitonin', 'Cholecystokinin', 'Cortisol', 'Dehydroepiandrosterone', 'Dihydrotestosterone', 'Erythropoietin', 'Estradiol', 'Oestradiol ', 'Estrogen', 'Estrone', 'Oestrone ', 'Gastrin', 'Ghrelin', 'Glucagon', 'Glucagon Like Peptide 1', 'Somatotrophin', 'Human chorionic Gonadotropin ', 'Insulin', 'Kisspeptin', 'metastin', 'Leptin', 'Luteinizing Hormone ', 'Melatonin', 'Oxytocin', 'Parathyrin ', 'Peptide YY', 'Progesterone', 'Prolactin', 'Prostaglandins', 'Relaxin', 'Serotonin', '5-hydroxytryptamine', 'Somatostatin', 'Testosterone', 'Triiodothyronin T3', 'Thyroxine T4', 'Vitamin D', 'Thyroid-stimulating hormone ', 'Thyrotropin ', 'Adrenocorticotropin', 'Follicle-stimulating hormone', 'Antidiuretic hormone', 'Vasopressin', 'Thyrotropin-releasing hormone', 'Growth hormone-releasing hormone', 'Corticotropin-releasing hormone', 'Gonadotropin-releasing hormone', 'Vascular Endothelial Growth Factor', 'Progestin

### Dataset analysis

In [ ]:
print(len(horm_gene))
print(type(horm_gene))
print(horm_gene['Adrenaline'])

59
<class 'dict'>
{'source': ['TH', 'CRH', 'ADRA2A', 'ADRA2C', 'CARTPT', 'ADRA2B', 'PNMT', 'VIP'], 'target': ['ADRB2', 'PDE4D', 'STAR', 'PPARGC1A', 'ADRA2B', 'ADRA1D', 'SIRT2', 'ADIPOQ', 'ADRA2A', 'RYR2', 'PDE4B', 'ADRB1', 'SNCA', 'ADRB3', 'PENK', 'KCNQ1', 'PKLR', 'SRD5A1', 'TPM1', 'ADRA2C', 'RNLS', 'ATP2B4', 'PRKACA', 'DRD5', 'ADRA1B', 'DRD4', 'SLC9A1', 'ADRA1A']}


In [ ]:
#To get count of genes for each hormone

gene_counts = {}
for hormone, genes in horm_gene.items():
    gene_counts[hormone] = [len(genes['source']), len(genes['target'])] 

### Retaining genes with embeddings

In [ ]:
with open('word_embeddings.txt') as json_file:
    initial_embeddings = json.load(json_file)

print(len(initial_embeddings))

1410


In [ ]:
#np_load_old = np.load()
#p.load = lambda a,*k: np_load_old(*a, allow_pickle=True, **k)
#extra_embeddings = np.load('word_vectors.npy')[()]
np_load_old = np.load

# modify the default parameters of np.load
np.load_new = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
#extra_embeddings = np.load('word_vectors.npy')



In [ ]:
extra_embeddings = np.load_new('word_vectors.npy')[()]
print(type(extra_embeddings))
print(extra_embeddings.keys())


<class 'dict'>
dict_keys(['rpl27', 'androgen', 'dehydroepiandrosterone', 'dihydrotestosterone', 'cst3', 'mbd1', 'hspd1', 'estrone', 'oestrone ', 'ffar4', 'glucagon like peptide 1', 'pick1', 'tns2', 'kisspeptin', 'metastin', 'lrp2', 'relaxin', 'mtap', 'slco1c1', 'progestin', 'aromatase'])


In [ ]:
len(horm_gene['Adrenaline']['source'])

8

In [ ]:
def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res

In [ ]:
embeddings = Merge(initial_embeddings, extra_embeddings)
len(embeddings.keys())

1431

In [ ]:
print(len(emb_horm_gene.keys()))
print(horm_gene['Adrenaline'])

NameError: ignored

In [ ]:
emb_gene_counts = {}
for hormone, genes in emb_horm_gene.items():
    emb_gene_counts[hormone] = [len(genes['source']), len(genes['target'])]
    #print(emb_gene_counts[hormone])


In [ ]:
### Preparing Source Embedding dataset of SMOTE

X_dataset_src = []
y_dataset_src = []

hormone_count = 0
count_list = []

y_map_src = {}
for hormone in horm_gene.keys():
    src_num_count = len(horm_gene[hormone]['source'])   
    tar_num_count = len(horm_gene[hormone]['target'])

    if src_num_count >= 6 and tar_num_count >= 6:           ### SMOTE requires each class to have atleast 6 examples. 
        y_map_src[hormone_count] = hormone                  ### So, consider hormones that have atleast 6 source and target genes
        count_list.append(src_num_count)
        if hormone in initial_embeddings.keys():
            for src_gene in horm_gene[hormone]['source']:

                if src_gene.lower() in initial_embeddings.keys():
                    source_gene = src_gene.lower()
                    X_dataset_src.append(initial_embeddings[source_gene])
                    y_dataset_src.append(hormone_count)

                elif src_gene.lower() in extra_embeddings.keys():
                    source_gene = src_gene.lower()
                    X_dataset_src.append(extra_embeddings[source_gene])
                    y_dataset_src.append(hormone_count)

            hormone_count = hormone_count + 1

        elif hormone.lower() in extra_embeddings.keys():
            for src_gene in horm_gene[hormone]['source']:

                if src_gene.lower() in initial_embeddings.keys():
                    source_gene = src_gene.lower()
                    X_dataset_src.append(initial_embeddings[source_gene])
                    y_dataset_src.append(hormone_count)

                elif src_gene.lower() in extra_embeddings.keys():
                    source_gene = src_gene.lower()
                    X_dataset_src.append(extra_embeddings[source_gene])
                    y_dataset_src.append(hormone_count)

            hormone_count = hormone_count + 1

print(y_dataset_src)
print(len(X_dataset_src[0]))
print(count_list)
print(X_dataset_src[0])
print(X_dataset_src[1])

[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 

In [ ]:
### Generate new embeddings for source genes using SMOTE

counter_src = Counter(y_dataset_src)
print(counter_src)
oversample_src = SMOTE()
X_dataset_src_oversampled, y_dataset_src_oversampled = oversample_src.fit_resample(np.array(X_dataset_src), np.array(y_dataset_src))
counter = Counter(y_dataset_src_oversampled)
print(counter)
print(type(X_dataset_src_oversampled))
print(len(X_dataset_src_oversampled))
print(len(y_dataset_src_oversampled))

Counter({7: 218, 10: 40, 19: 36, 4: 32, 20: 24, 14: 23, 15: 23, 8: 21, 11: 20, 12: 20, 3: 16, 6: 16, 2: 15, 16: 12, 5: 10, 9: 10, 13: 9, 0: 8, 1: 8, 17: 7, 18: 7, 21: 6})
Counter({0: 218, 1: 218, 2: 218, 3: 218, 4: 218, 5: 218, 6: 218, 7: 218, 8: 218, 9: 218, 10: 218, 11: 218, 12: 218, 13: 218, 14: 218, 15: 218, 16: 218, 17: 218, 18: 218, 19: 218, 20: 218, 21: 218})
<class 'numpy.ndarray'>
4796
4796


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [ ]:
### Prepare source dataset, a dictionary of this format:  hormone_name -> 216 samples, each sample being an embedding of 200 dimensions 

source_dict = {}
for hormone, embedding in zip(y_dataset_src_oversampled, X_dataset_src_oversampled):
    if y_map_src[hormone] in source_dict:
        source_dict[y_map_src[hormone]].append(list(embedding))
    
    else:
        source_dict[y_map_src[hormone]] = [list(embedding)]

In [ ]:
len(source_dict.keys())

22

In [ ]:
X_dataset_tar = []
y_dataset_tar = []

hormone_count = 0
count_list = []

y_map_tar = {}
for hormone in horm_gene.keys():
    src_num_count = len(horm_gene[hormone]['source'])   
    tar_num_count = len(horm_gene[hormone]['target'])

    if src_num_count >= 6 and tar_num_count>=6:
        y_map_tar[hormone_count] = hormone
        count_list.append(tar_num_count)
        if hormone in initial_embeddings.keys():
            for tar_gene in horm_gene[hormone]['target']:

                if tar_gene.lower() in initial_embeddings.keys():
                    target_gene = tar_gene.lower()
                    X_dataset_tar.append(initial_embeddings[target_gene])
                    y_dataset_tar.append(hormone_count)

                elif tar_gene.lower() in extra_embeddings.keys():
                    target_gene = tar_gene.lower()
                    X_dataset_tar.append(extra_embeddings[target_gene])
                    y_dataset_tar.append(hormone_count)

            hormone_count = hormone_count + 1

        elif hormone.lower() in extra_embeddings.keys():
            for tar_gene in horm_gene[hormone]['target']:

                if tar_gene.lower() in initial_embeddings.keys():
                    target_gene = tar_gene.lower()
                    X_dataset_tar.append(initial_embeddings[target_gene])
                    y_dataset_tar.append(hormone_count)

                elif tar_gene.lower() in extra_embeddings.keys():
                    target_gene = tar_gene.lower()
                    X_dataset_tar.append(extra_embeddings[target_gene])
                    y_dataset_tar.append(hormone_count)

            hormone_count = hormone_count + 1

print(y_dataset_tar)
print(len(X_dataset_tar[0]))
print(count_list)
print(X_dataset_tar[0])
print(X_dataset_tar[1])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 

In [ ]:
counter_tar = Counter(y_dataset_tar)
print(counter_tar)
oversample_tar = SMOTE()
X_dataset_tar_oversampled, y_dataset_tar_oversampled = oversample_tar.fit_resample(np.array(X_dataset_tar), np.array(y_dataset_tar))
counter = Counter(y_dataset_tar_oversampled)
print(counter)
print(type(X_dataset_tar_oversampled))
print(len(X_dataset_tar_oversampled))
print(len(y_dataset_tar_oversampled))

Counter({7: 300, 4: 153, 19: 150, 11: 58, 12: 58, 8: 51, 16: 51, 13: 40, 6: 37, 10: 37, 5: 36, 0: 28, 1: 28, 18: 19, 20: 19, 14: 14, 15: 12, 9: 8, 21: 8, 2: 7, 3: 6, 17: 6})
Counter({0: 300, 1: 300, 2: 300, 3: 300, 4: 300, 5: 300, 6: 300, 7: 300, 8: 300, 9: 300, 10: 300, 11: 300, 12: 300, 13: 300, 14: 300, 15: 300, 16: 300, 17: 300, 18: 300, 19: 300, 20: 300, 21: 300})
<class 'numpy.ndarray'>
6600
6600


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarni

In [ ]:
target_dict = {}
for hormone, embedding in zip(y_dataset_tar_oversampled, X_dataset_tar_oversampled):
    if y_map_tar[hormone] in target_dict:
        target_dict[y_map_tar[hormone]].append(list(embedding))
    
    else:
        target_dict[y_map_tar[hormone]] = [list(embedding)]

## Random Sampling

##### SMOTE generates 216 gene samples for homone source and 299 gene samples for hormone targers. Since we need equal number of source and target genes for each hormone, I randomly sampled 216 genes from target genes. 

In [ ]:
for hormone in target_dict:
    random_choices = np.random.choice(len(target_dict[hormone]), len(source_dict[hormone]))
    
    target_dict[hormone] = [list(target_dict[hormone][sample]) for sample in random_choices]
    

In [ ]:
len(source_dict['Adrenaline']) == len(target_dict['Adrenaline'])

True

In [ ]:
dataset = {}
dataset['source'] = source_dict
dataset['target'] = target_dict

print(dataset.keys())
print('Number of hormones')
print(len(dataset['source']))
print(len(dataset['target']))
print('Number of samples for a hormone')
print(len(dataset['source']['Adrenaline']))
print(len(dataset['target']['Adrenaline']))
print('Dimension of embedding for each sample')
print(len(dataset['source']['Adrenaline'][0]))
print(len(dataset['target']['Adrenaline'][0]))

dict_keys(['source', 'target'])
Number of hormones
22
22
Number of samples for a hormone
218
218
Dimension of embedding for each sample
200
200


In [ ]:
with open('SMOTE_oversampled_dataset.json', 'w') as fp:
    json.dump(dataset, fp)


no_embedding = []
emb_horm_gene = {}
embedding_list = list(embeddings.keys())

for hormone, genes in horm_gene.items():
    if hormone in embedding_list:

        source_list = []
        target_list = []

        for source_gene in genes['source']:
            if source_gene.lower() in embedding_list:
                source_list.append(source_gene.lower())

            elif source_gene.lower() not in no_embedding:
                print(source_gene.lower())
                no_embedding.append(source_gene.lower())

        for target_gene in genes['target']:
            if target_gene.lower() in embedding_list:
                target_list.append(target_gene.lower())

            elif target_gene.lower() not in no_embedding:
                print(target_gene.lower())
                no_embedding.append(target_gene.lower())

        emb_horm_gene[hormone] = {'source' : source_list, 'target' : target_list}

    else:
        print(hormone)
        no_embedding.append(hormone.lower())



with open('hormone_gene_graph0.txt') as json_file:
    graph = json.load(json_file)

hormone_list = graph['pos'].keys()
hormone_list

for horm in hormone_list:
    print(len(graph['pos'][horm]), len(graph['neg'][horm]))

print(graph['pos'].keys())
print(len(graph['pos']['Cortisol']))

X_dataset_pos = []
y_dataset_pos = []

hormone_count = 0
count_list = []

y_map = {}
for hormone in graph['pos'].keys():
    num_count = len(graph['pos'][hormone])   
    
    if num_count >= 6:
        y_map[hormone_count] = hormone
        count_list.append(num_count)
        for pos_gene in graph['pos'][hormone]:
            
            X_dataset_pos.append(embeddings[hormone] + embeddings[pos_gene])
            y_dataset_pos.append(hormone_count)

        

    
        hormone_count = hormone_count + 1

print(y_dataset_pos)
print(len(X_dataset_pos[0]))
print(count_list)
print(X_dataset_pos[0])
print(X_dataset_pos[1])

counter_pos = Counter(y_dataset_pos)
print(counter_pos)
oversample_pos = SMOTE()
X_dataset_pos_oversampled, y_dataset_pos_oversampled = oversample_pos.fit_resample(np.array(X_dataset_pos), np.array(y_dataset_pos))
counter = Counter(y_dataset_pos_oversampled)
print(type(X_dataset_pos_oversampled))
print(len(X_dataset_pos_oversampled))
print(len(y_dataset_pos_oversampled))

X_dataset_neg = []
y_dataset_neg = []

hormone_count = 0
count_list = []

y_map = {}
for hormone in graph['neg'].keys():
    num_count = len(graph['neg'][hormone])   
    
    if num_count >= 6:
        y_map[hormone_count] = hormone
        count_list.append(num_count)
        for pos_gene in graph['neg'][hormone]:
            
            X_dataset_neg.append(embeddings[hormone] + embeddings[pos_gene])
            y_dataset_neg.append(hormone_count)

        

    
        hormone_count = hormone_count + 1

print(y_dataset_neg)
print(len(X_dataset_neg[0]))
print(count_list)
print(X_dataset_neg[0])
print(X_dataset_neg[1])

counter_neg = Counter(y_dataset_neg)
print(counter_neg)
oversample_neg = SMOTE()
X_dataset_neg_oversampled, y_dataset_neg_oversampled = oversample_neg.fit_resample(np.array(X_dataset_neg), np.array(y_dataset_neg))
counter = Counter(y_dataset_neg_oversampled)
print(type(X_dataset_neg_oversampled))
print(len(X_dataset_neg_oversampled))
print(len(y_dataset_neg_oversampled))

X_train = {}
temp = {}

#x_counter = 0
for i,j in zip(y_dataset_pos_oversampled, X_dataset_pos_oversampled):
    if y_map[i] not in temp.keys():
        temp[y_map[i]] = []

    temp[y_map[i]].append(j)
    #x_counter += 1

X_train['pos'] = temp

temp = {}
#x_counter = 0
for i,j in zip(y_dataset_neg_oversampled, X_dataset_neg_oversampled):
    if y_map[i] not in temp.keys():
        temp[y_map[i]] = []

    temp[y_map[i]].append(j)
    #x_counter += 1

X_train['neg'] = temp

print(X_train['pos']['Erythropoietin'])
print(X_train['neg']['Erythropoietin'])